In [2]:
import pandas as pd
import numpy as np
from thefuzz import process
import imdb
import fuzzyset

In [3]:
films = pd.read_csv('./data/ml-25m/ml-25m/movies.csv')
films.drop(columns=['genres'], inplace=True)

In [18]:
ratings = pd.read_csv('./data/ml-25m/ml-25m/ratings.csv')
ratings.drop(columns=['timestamp'], inplace=True)

In [19]:
films.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [20]:
#removing the release year and making it its own column
films['release_year'] =  films['title'].apply(lambda x: x[-5:-1])

# removing parentheticals from titles for searching purposes:
films['primary_title'] = films['title'].apply(lambda x: x.split('(')[0][:-1])

#fixing titles that have "the" moved to the end for matching purposes
test_list = []
for title in films['primary_title']:
    if str(title[-3:]) == 'The':
        test_list.append('The ' + str(title[:-5]))
    else:
        test_list.append(title)
films['primary_title'] = test_list

In [21]:
films.head()

,movieId,title,release_year,primary_title
0,1,Toy Story (1995),1995,Toy Story
1,2,Jumanji (1995),1995,Jumanji
2,3,Grumpier Old Men (1995),1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),1995,Father of the Bride Part II


In [22]:
movies = pd.read_csv('./data/data_for_eda.csv')

In [23]:
movies.head()

,Unnamed: 0,tconst,primary_title,release_year,runtime,rating,votes,tomato_score,metacritic_score,mpaa_rating,...,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7,cast_8,cast_9,cast_10,plot_polarity
0,0,tt0111161,The Shawshank Redemption,1994,142,9.3,2540263,91%,80/100,R,...,209.0,151.0,348409.0,6669.0,1104.0,175.0,555550.0,2353.0,5683.0,0.018280
1,1,tt0468569,The Dark Knight,2008,152,9.0,2490570,94%,84/100,PG-13,...,288.0,5132.0,1173.0,323.0,634240.0,634300.0,275286.0,4170.0,650038.0,0.042357
2,2,tt1375666,Inception,2010,148,8.8,2231336,87%,74/100,PG-13,...,138.0,330687.0,680983.0,913822.0,634240.0,858799.0,1877.0,2892.0,809059.0,0.029848
3,3,tt0137523,Fight Club,1999,139,8.8,1999188,79%,66/100,R,...,93.0,1570.0,1533.0,340260.0,399.0,657333.0,880243.0,68501.0,149556.0,-0.008166
4,4,tt0109830,Forrest Gump,1994,142,8.8,1960064,70%,82/100,PG-13,...,158.0,705.0,641.0,398.0,709.0,343165.0,744839.0,277704.0,823330.0,0.079285


In [24]:
#for films, adding the year back into the title to help with matching efficiency
match_title = []
title_list = films['primary_title']
year_list = films['release_year']
for i in range(0,len(title_list)):
    match_title.append(title_list[i] + ' (' + year_list[i] + ')')
    
films['match_title'] = match_title

#for movies, adding the year back into the title to help with matching efficiency
match_title = []
title_list = movies['primary_title']
year_list = movies['release_year']
for i in range(0,len(title_list)):
    match_title.append(title_list[i] + ' (' + str(year_list[i]) + ')')
    
movies['match_title'] = match_title

In [25]:
movies.head()

,Unnamed: 0,tconst,primary_title,release_year,runtime,rating,votes,tomato_score,metacritic_score,mpaa_rating,...,cast_3,cast_4,cast_5,cast_6,cast_7,cast_8,cast_9,cast_10,plot_polarity,match_title
0,0,tt0111161,The Shawshank Redemption,1994,142,9.3,2540263,91%,80/100,R,...,151.0,348409.0,6669.0,1104.0,175.0,555550.0,2353.0,5683.0,0.018280,The Shawshank Redemption (1994)
1,1,tt0468569,The Dark Knight,2008,152,9.0,2490570,94%,84/100,PG-13,...,5132.0,1173.0,323.0,634240.0,634300.0,275286.0,4170.0,650038.0,0.042357,The Dark Knight (2008)
2,2,tt1375666,Inception,2010,148,8.8,2231336,87%,74/100,PG-13,...,330687.0,680983.0,913822.0,634240.0,858799.0,1877.0,2892.0,809059.0,0.029848,Inception (2010)
3,3,tt0137523,Fight Club,1999,139,8.8,1999188,79%,66/100,R,...,1570.0,1533.0,340260.0,399.0,657333.0,880243.0,68501.0,149556.0,-0.008166,Fight Club (1999)
4,4,tt0109830,Forrest Gump,1994,142,8.8,1960064,70%,82/100,PG-13,...,705.0,641.0,398.0,709.0,343165.0,744839.0,277704.0,823330.0,0.079285,Forrest Gump (1994)


In [26]:
films.head()

,movieId,title,release_year,primary_title,match_title
0,1,Toy Story (1995),1995,Toy Story,Toy Story (1995)
1,2,Jumanji (1995),1995,Jumanji,Jumanji (1995)
2,3,Grumpier Old Men (1995),1995,Grumpier Old Men,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),1995,Waiting to Exhale,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),1995,Father of the Bride Part II,Father of the Bride Part II (1995)


In [27]:
#finding out which films need to be fuzzy joined, since there are some direct matches
movies = pd.merge(movies, films, on='primary_title', how='left')
movies.isnull().sum()
#there are 2,125 that do not have a direct title match. we will subset accordingly

Unnamed: 0             0
tconst                 0
primary_title          0
release_year_x         0
runtime                0
rating                 0
votes                  0
tomato_score        1067
metacritic_score    2688
mpaa_rating          381
plot                   0
genre_1                0
genre_2             1086
genre_3             4289
director_1             0
director_2          1086
director_3          4289
writer_1               0
writer_2            3722
writer_3            7963
cast_1                 0
cast_2                 2
cast_3                11
cast_4                11
cast_5                14
cast_6                17
cast_7                35
cast_8                77
cast_9               196
cast_10              590
plot_polarity          0
match_title_x          0
movieId             1490
title               1490
release_year_y      1490
match_title_y       1490
dtype: int64

In [28]:
# movies.drop(columns=['release_year_y', 'match_title_y'], inplace=True)

In [29]:
movies.dtypes

Unnamed: 0            int64
tconst               object
primary_title        object
release_year_x        int64
runtime               int64
rating              float64
votes                 int64
tomato_score         object
metacritic_score     object
mpaa_rating          object
plot                 object
genre_1              object
genre_2              object
genre_3              object
director_1           object
director_2           object
director_3           object
writer_1             object
writer_2             object
writer_3             object
cast_1                int64
cast_2              float64
cast_3              float64
cast_4              float64
cast_5              float64
cast_6              float64
cast_7              float64
cast_8              float64
cast_9              float64
cast_10             float64
plot_polarity       float64
match_title_x        object
movieId             float64
title                object
release_year_y       object
match_title_y       

In [30]:
movies['release_year_x'] = movies['release_year_x'].apply(lambda x: str(x))

In [31]:
movies.head()

,Unnamed: 0,tconst,primary_title,release_year_x,runtime,rating,votes,tomato_score,metacritic_score,mpaa_rating,...,cast_7,cast_8,cast_9,cast_10,plot_polarity,match_title_x,movieId,title,release_year_y,match_title_y
0,0,tt0111161,The Shawshank Redemption,1994,142,9.3,2540263,91%,80/100,R,...,175.0,555550.0,2353.0,5683.0,0.018280,The Shawshank Redemption (1994),318.0,"Shawshank Redemption, The (1994)",1994,The Shawshank Redemption (1994)
1,1,tt0468569,The Dark Knight,2008,152,9.0,2490570,94%,84/100,PG-13,...,634300.0,275286.0,4170.0,650038.0,0.042357,The Dark Knight (2008),58559.0,"Dark Knight, The (2008)",2008,The Dark Knight (2008)
2,1,tt0468569,The Dark Knight,2008,152,9.0,2490570,94%,84/100,PG-13,...,634300.0,275286.0,4170.0,650038.0,0.042357,The Dark Knight (2008),130219.0,The Dark Knight (2011),2011,The Dark Knight (2011)
3,2,tt1375666,Inception,2010,148,8.8,2231336,87%,74/100,PG-13,...,858799.0,1877.0,2892.0,809059.0,0.029848,Inception (2010),79132.0,Inception (2010),2010,Inception (2010)
4,3,tt0137523,Fight Club,1999,139,8.8,1999188,79%,66/100,R,...,657333.0,880243.0,68501.0,149556.0,-0.008166,Fight Club (1999),2959.0,Fight Club (1999),1999,Fight Club (1999)


In [32]:
movies_fuzzy = movies[movies['movieId'].isnull() == True]

In [33]:
films.head()

,movieId,title,release_year,primary_title,match_title
0,1,Toy Story (1995),1995,Toy Story,Toy Story (1995)
1,2,Jumanji (1995),1995,Jumanji,Jumanji (1995)
2,3,Grumpier Old Men (1995),1995,Grumpier Old Men,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),1995,Waiting to Exhale,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),1995,Father of the Bride Part II,Father of the Bride Part II (1995)


In [34]:
title_list = fuzzyset.FuzzySet()
for title in films['match_title']:
    title_list.add(title)
    
match_list = []
for film_title in movies['primary_title']:
    match_list.append(title_list.get(film_title)[0][1])
    
movies['match_list'] = match_list

In [35]:
movies.head()

,Unnamed: 0,tconst,primary_title,release_year_x,runtime,rating,votes,tomato_score,metacritic_score,mpaa_rating,...,cast_8,cast_9,cast_10,plot_polarity,match_title_x,movieId,title,release_year_y,match_title_y,match_list
0,0,tt0111161,The Shawshank Redemption,1994,142,9.3,2540263,91%,80/100,R,...,555550.0,2353.0,5683.0,0.018280,The Shawshank Redemption (1994),318.0,"Shawshank Redemption, The (1994)",1994,The Shawshank Redemption (1994),The Shawshank Redemption (1994)
1,1,tt0468569,The Dark Knight,2008,152,9.0,2490570,94%,84/100,PG-13,...,275286.0,4170.0,650038.0,0.042357,The Dark Knight (2008),58559.0,"Dark Knight, The (2008)",2008,The Dark Knight (2008),The Dark Knight (2008)
2,1,tt0468569,The Dark Knight,2008,152,9.0,2490570,94%,84/100,PG-13,...,275286.0,4170.0,650038.0,0.042357,The Dark Knight (2008),130219.0,The Dark Knight (2011),2011,The Dark Knight (2011),The Dark Knight (2008)
3,2,tt1375666,Inception,2010,148,8.8,2231336,87%,74/100,PG-13,...,1877.0,2892.0,809059.0,0.029848,Inception (2010),79132.0,Inception (2010),2010,Inception (2010),Inception (2010)
4,3,tt0137523,Fight Club,1999,139,8.8,1999188,79%,66/100,R,...,880243.0,68501.0,149556.0,-0.008166,Fight Club (1999),2959.0,Fight Club (1999),1999,Fight Club (1999),Fight Club (1999)


In [36]:
#getting a list of the films that have been properly matched

year_list_1 = list(movies['match_title_x'])
year_list_2 = list(movies['match_list'])
titles = list(movies['primary_title'])
matched = []

for i in range(0, len(year_list_1)):
    if year_list_1[i][-6:] == year_list_2[i][-6:]:
        matched.append(titles[i])

In [37]:
#reducing the dataset to just movies that have been paired properly
movies = movies[movies['primary_title'].isin(matched)==True]

#we may have some duplicates but can deal with those later. 

In [38]:
#subsetting to keep only movies where the match_list title year matches the release year

prim_title_year = list(movies['release_year_x'])
match_list_year = []
for title in movies['match_list']:
    match_list_year.append(title[-5:-1])

In [39]:
#we have 9091 titles that have reviews that we can match them to. 
len(matched)

9091

In [40]:
matched_movies = []
primary_title = list(movies['primary_title'])
for i in range(0, len(primary_title)):
    if prim_title_year[i] == match_list_year[i]:
        matched_movies.append(primary_title[i])

In [41]:
movies = movies[movies['primary_title'].isin(matched_movies) == True]

In [42]:
#subsetting to keep only movies where the match_list title actually matches the primary_title
prim_title_year = list(movies['release_year_x'])
match_list_year = []
for title in movies['match_list']:
    match_list_year.append(title[-5:-1])
matched_movies = []

primary_title = list(movies['primary_title'])
for i in range(0, len(primary_title)):
    if prim_title_year[i] == match_list_year[i]:
        matched_movies.append(primary_title[i])

In [43]:
movies = movies[movies['primary_title'].isin(matched_movies)==True]

In [44]:
movies.drop_duplicates(subset='match_title_x', inplace=True)

In [51]:
#getting just the matching title and movie ID so we can add the ID to our movies dataset. 
films2 = pd.read_csv('./data/ml-25m/ml-25m/movies.csv')

#removing the release year and making it its own column
films2['release_year'] =  films2['title'].apply(lambda x: x[-5:-1])

# removing parentheticals from titles for searching purposes:
films2['primary_title'] = films2['title'].apply(lambda x: x.split('(')[0][:-1])

#fixing titles that have "the" moved to the end for matching purposes
test_list = []
for title in films2['primary_title']:
    if str(title[-3:]) == 'The':
        test_list.append('The ' + str(title[:-5]))
    else:
        test_list.append(title)
films2['primary_title'] = test_list

#for films, adding the year back into the title to help with matching efficiency
match_title = []
title_list = films2['primary_title']
year_list = films2['release_year']
for i in range(0,len(title_list)):
    match_title.append(title_list[i] + ' (' + year_list[i] + ')')
    
films2['match_title'] = match_title

films2.drop(columns=['title', 'genres', 'release_year'], inplace=True) 

films2.rename(columns={'match_title':'match_list'}, inplace=True)

In [53]:
movies.drop(columns=['movieId', 'title', 'release_year_y', 'match_title_y', 'match_title_x'], inplace=True)

In [55]:
movies = pd.merge(movies, films2, how='left', on='match_list')

In [58]:
movies.drop(columns=['match_list', 'primary_title_y', 'match_list'], inplace=True)

In [60]:
movies.to_csv('./data/movies_with_review_id.csv', index=False)